In [1]:
from cluster_model import get_dataset, get_transformer, DATASET_NAME, MODEL_NAME, SEED, N_DATASIZE, get_block_out_label, N_BLOCKS
import cluster_model

ds = get_dataset(DATASET_NAME)
model, _ = get_transformer(MODEL_NAME)
shuffled = ds.shuffle(seed=SEED)['train'][:N_DATASIZE]['text']

ds = shuffled
labs = [get_block_out_label(i) for i in range(N_BLOCKS)]

/home/lev/code/research/ai/lattice_cluster_paths/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


running k-means on cuda:0..


[running kmeans]: 8it [00:00, 43.04it/s, center_shift=0.000066, iteration=8, tol=0.000100]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded pretrained model EleutherAI/pythia-70m into HookedTransformer
Moving model to device:  cuda


In [2]:
import importlib
importlib.reload(cluster_model)

decomp = cluster_model.Decomposer(model, ds, labs, similarity_cutoff=2)
decomp.load()

Loading dataset from cache to device cuda
Loading dataset from cache to device cuda
Loading dataset from cache to device cuda
Finding optimal number of clusters and such clusters for layer blocks.0.hook_resid_post
Loading GMM clusters from cache
Finding optimal number of clusters and such clusters for layer blocks.1.hook_resid_post
Loading GMM clusters from cache
Finding optimal number of clusters and such clusters for layer blocks.2.hook_resid_post
Loading GMM clusters from cache
Loading cluster scores from cache


In [3]:
import utils
importlib.reload(utils)
utils.top_k_dag_paths_dynamic(decomp.lattice_scores, k=200_000)

[([26, 16, 15, 0], 160.29613049194995),
 ([26, 16, 12, 0], 104.42872031504885),
 ([26, 16, 23, 0], 102.62855330400635),
 ([26, 16, 26, 0], 102.6263009646791),
 ([26, 16, 27, 0], 102.6201057961589),
 ([26, 16, 5, 0], 102.61412677548651),
 ([26, 16, 10, 0], 102.61371881004015),
 ([26, 16, 29, 0], 102.61116641981062),
 ([26, 16, 18, 0], 102.61025958605751),
 ([26, 16, 13, 0], 102.60850326229411),
 ([26, 16, 8, 0], 102.60205890213547),
 ([26, 16, 22, 0], 102.59822462721786),
 ([26, 16, 17, 0], 102.59710300245206),
 ([26, 16, 6, 0], 102.59507339783886),
 ([26, 16, 7, 0], 102.59438683652843),
 ([26, 16, 4, 0], 102.59366242673423),
 ([26, 16, 9, 0], 102.5934856365493),
 ([26, 16, 25, 0], 102.58621707382554),
 ([26, 16, 2, 0], 102.5859588606836),
 ([26, 16, 19, 0], 102.58387526300794),
 ([26, 16, 21, 0], 102.57723914257076),
 ([26, 16, 16, 0], 102.57595000580477),
 ([26, 16, 14, 0], 102.57439685886493),
 ([26, 16, 0, 0], 102.57401620660676),
 ([26, 16, 24, 0], 102.56054537688033),
 ([26, 16, 1

In [4]:

score_path = [8, 57, 89]
score_path = [10, 63, 89]
score_path = [3, 3, 1]
score_path = [26, 16, 27]
"""
Paths which seem to have meaning

[1, 3, 25]: relational words like "if" "in" "soon"
[1, 3, 10]: relational word but now it is more specific to "to"
"""

to_score = [d[:100] for d in ds][:100]
scores = decomp.score(
    to_score,
    score_path=score_path
)

DS (3, 2816, 512) 2816
(2816, 3, 512) [5, 4, 6]


In [28]:
import math
import numpy as np

scores_per_token_set = np.array([max(s) for s in scores])
top_args = np.argsort(scores_per_token_set)[::-1]

tokens = [[model.tokenizer.decode(t) for t in model.tokenizer(d)[
    'input_ids']] for d in to_score]
tokens_reord = [tokens[i] for i in top_args]
scores_reord = [scores[i] for i in top_args]
# TODO: SORT!

# TODO sep fun
from circuitsvis.utils.render import RenderedHTML, render
html = render(
    "TextNeuronActivations",
    tokens=tokens_reord,
    activations=[[[[0 if math.isnan(tok * 100) else int(math.exp(tok * 100))]] for tok in s] for s in scores_reord],
    firstDimensionName="Layer",
    secondDimensionName="Neuron",
    firstDimensionLabels=None,
    secondDimensionLabels=None
)

In [29]:
scores_per_token_set, scores_per_token_set[top_args]

(array([0.13637187, 0.13354415, 0.13592945, 0.12109772, 0.11476188,
        0.12774346, 0.11390934, 0.13107547, 0.11347243, 0.1385617 ,
        0.13279616, 0.14094906, 0.11740224, 0.1170346 , 0.13828102,
        0.10986638, 0.1305698 , 0.12940961, 0.131775  , 0.11676633,
        0.11583677, 0.12513784, 0.11627351, 0.12658045, 0.13429993,
        0.13260536, 0.13607731, 0.12935323, 0.13781472, 0.11904823,
        0.11342997, 0.13522786, 0.14152704, 0.11215523, 0.11194874,
        0.12714779, 0.11198738, 0.12168759, 0.13610801, 0.12948263,
        0.13582978, 0.12109772, 0.11746155, 0.13808872, 0.11894938,
        0.11173074, 0.13844902, 0.11004403, 0.13003742, 0.11356048,
        0.13338256, 0.11102356, 0.13486474, 0.13404343, 0.12880257,
        0.11644961, 0.1359249 , 0.12388298, 0.13136499, 0.11813175,
        0.11120583, 0.12415985, 0.13593426, 0.1165956 , 0.12893824,
        0.13637534, 0.13715039, 0.13852351, 0.11757725, 0.1174936 ,
        0.13146668, 0.10832196, 0.13601411, 0.11

In [30]:
from IPython.core.display import display, HTML

display(HTML(str(html)))

/tmp/ipykernel_280005/288767411.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML
